In [ ]:
import htcondor
import ipywidgets as widgets
from IPython.display import display
import os

# ─── 1) Core script + args ───────────────────────────────────────────
script = widgets.Text(
    value='',
    description='Script path:',
    placeholder='/home/you/project/my_job.sh',
    layout=widgets.Layout(width='60%')
)
script.style.description_width = '30%'

args = widgets.Text(
    value='',
    description='Arguments:',
    layout=widgets.Layout(width='60%')
)
args.style.description_width = '30%'

# ─── 2) Resource requests ────────────────────────────────────────────
cpus = widgets.IntSlider(
    value=1, min=1, max=32,
    description='CPUs:'
)
cpus.style.description_width = '30%'
cpus.layout.width = '60%'

mem = widgets.IntText(
    value=1024,
    description='Memory (MB):'
)
mem.style.description_width = '30%'
mem.layout.width = '60%'

disk = widgets.IntText(
    value=1024,
    description='Disk (MB):'
)
disk.style.description_width = '30%'
disk.layout.width = '60%'

gpus = widgets.IntSlider(
    value=0, min=0, max=4,
    description='GPUs:'
)
gpus.style.description_width = '30%'
gpus.layout.width = '60%'

# ─── 3) Environment & universe ───────────────────────────────────────
universe = widgets.Dropdown(
    options=['vanilla','docker','singularity'],
    value='vanilla',
    description='Universe:'
)
universe.style.description_width = '30%'
universe.layout.width = '60%'

docker_image = widgets.Text(
    value='',
    description='Docker image:',
    placeholder='e.g. pytorch/pytorch:latest',
    layout=widgets.Layout(width='60%')
)
docker_image.style.description_width = '30%'

# only show docker_image when universe=='docker'
def on_uni_change(change):
    docker_image.layout.display = 'block' if change['new']=='docker' else 'none'
universe.observe(on_uni_change, names='value')
docker_image.layout.display = 'none'

# ─── 4) Advanced: requirements & transfer files ─────────────────────
requirements = widgets.Text(
    value='',
    description='Requirements:',
    placeholder='e.g. OpSys=="LINUX"',
    layout=widgets.Layout(width='60%')
)
requirements.style.description_width = '30%'

transfer_in = widgets.Text(
    value='',
    description='Transfer in:',
    placeholder='file1.txt,file2.dat',
    layout=widgets.Layout(width='60%')
)
transfer_in.style.description_width = '30%'

# ─── 5) Buttons + status output ───────────────────────────────────────
submit_btn = widgets.Button(description='Submit Job', button_style='primary')
preview_btn = widgets.Button(description='Preview Job', button_style='info')

button_row = widgets.HBox([submit_btn, preview_btn])

status_out = widgets.Output(layout=widgets.Layout(width='60%'))
status_box = widgets.VBox([status_out])
status_box.layout = widgets.Layout(
    width='60%',
    min_height='4rem',
    margin='1rem 0 0 0',
    border='1px solid #ddd',
    padding='0.5rem',
    border_radius='4px'
)

# ─── 6) Assemble & center ────────────────────────────────────────────
card = widgets.VBox(
    [
      widgets.HTML("<h1 style='text-align:center;'>HTCondor Submission</h1>"),
      widgets.HTML("<p style='text-align:center;color:#555;'>Fill in all fields then click Submit.</p>"),
      script, args,
      cpus, mem, disk, gpus,
      universe, docker_image,
      requirements, transfer_in,
      button_row,
      status_box
    ],
    layout=widgets.Layout(
        width='80%', max_width='800px', padding='2%',
        border='1px solid #ddd', border_radius='8px',
        display='flex', flex_flow='column', align_items='center'
    )
)

container = widgets.Box(
    [card],
    layout=widgets.Layout(
        display='flex', align_items='center',
        justify_content='center', height='100vh', width='100vw'
    )
)

display(container)

# ─── 7) Submission handler ───────────────────────────────────────────
def build_submit_dict():
    path = script.value.strip()
    if not path:
        raise ValueError("No script path provided.")
    if not os.path.isabs(path):
        path = os.path.abspath(path)

    submit_dict = {
        'executable': path,
        'arguments': args.value,
        'request_cpus': str(cpus.value),
        'request_memory': f"{mem.value}MB",
        'request_disk': f"{disk.value}MB",
        'request_gpus': str(gpus.value),
        'universe': universe.value,
        'output': '$(Cluster).$(Process).out',
        'error': '$(Cluster).$(Process).err',
        'log': '$(Cluster).log'
    }

    if universe.value == 'docker' and docker_image.value.strip():
        submit_dict['docker_image'] = docker_image.value.strip()
    if requirements.value.strip():
        submit_dict['requirements'] = requirements.value.strip()
    if transfer_in.value.strip():
        submit_dict['should_transfer_files'] = 'YES'
        submit_dict['transfer_input_files'] = transfer_in.value.strip()

    return submit_dict

def on_submit(b):
    b.disabled = True
    preview_btn.disabled = True
    status_out.clear_output()
    with status_out:
        print("Queuing…")
    try:
        submit_dict = build_submit_dict()
        if not os.path.exists(submit_dict['executable']):
            raise FileNotFoundError(f"`{submit_dict['executable']}` not found.")

        schedd = htcondor.Schedd()
        desc = htcondor.Submit(submit_dict)
        with schedd.transaction() as txn:
            cid = desc.queue(txn)
        with status_out:
            print(f"Submitted as cluster {cid}")
    except Exception as e:
        with status_out:
            print(f"Failed: {e}")
    finally:
        b.disabled = False
        preview_btn.disabled = False

def on_preview(b):
    submit_btn.disabled = True
    b.disabled = True
    status_out.clear_output()
    try:
        submit_dict = build_submit_dict()
        submit = htcondor.Submit(submit_dict)
        with status_out:
            print("Preview of HTCondor Submit Description:")
            print("-" * 40)
            print(str(submit))
            print("-" * 40)
            print("This is not yet submitted.")
    except Exception as e:
        with status_out:
            print(f"Preview failed: {e}")
    finally:
        submit_btn.disabled = False
        b.disabled = False

submit_btn.on_click(on_submit)
preview_btn.on_click(on_preview)

